In [ ]:
import pathlib
import imageio
import numpy as np

training_paths = pathlib.Path('D:/Projects/starik/stage1_train').glob('*/images/*.png')
training_sorted = sorted([x for x in training_paths])
im_path = training_sorted[146]
im = imageio.imread(str(im_path))

In [2]:
print('Original shape: {}'.format(im.shape))

from skimage.color import rgb2gray
im_gray = rgb2gray(im)
print('New shape: {}'.format(im_gray.shape))

Original shape: (256, 320, 4)


New shape: (256, 320)


In [3]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(im)
plt.axis('off')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(im_gray, cmap='gray')
plt.axis('off')
plt.title('Grayscale')

plt.tight_layout()
plt.show()


In [4]:
from skimage.filters import threshold_otsu

#using Otsu method to get threshold value

thresh_val = threshold_otsu(im_gray)
mask = np.where(im_gray > thresh_val, 1, 0)

if np.sum(mask == 0) < np.sum(mask == 1):
    mask = np.where(mask, 0, 1)


In [5]:
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
im_pixels = im_gray.flatten()
plt.hist(im_pixels, bins=50, color="purple")
plt.vlines(thresh_val, 0, 100000, linestyle='--')
plt.ylim([0, 50000])
plt.title('Histogram')

plt.subplot(1, 2, 2)
mask_for_display = np.where(mask, mask, np.nan)
plt.imshow(im_gray, cmap='gray')
plt.imshow(mask_for_display, cmap='autumn', alpha=0.5)
plt.axis('off')
plt.title('Masked')

plt.show()


In [7]:
from scipy import ndimage
labels, nlabels = ndimage.label(mask)

label_arrays = []
for label_num in range(1, nlabels+1):
    label_mask = np.where(labels == label_num, 1, 0)
    label_arrays.append(label_mask)

print('{} objects was found.'.format(nlabels))

43 objects was found.


In [8]:
from matplotlib.colors import ListedColormap

color_map = ListedColormap(np.random.rand(256, 3))

labels_for_display = np.where(labels > 0, labels, np.nan)
plt.imshow(im_gray, cmap='gray')
plt.imshow(labels_for_display, cmap=color_map)
plt.axis('off')
plt.title('Separate cells ({} nuclei)'.format(nlabels))
plt.show()


In [10]:
for label_ind, label_coords in enumerate(ndimage.find_objects(labels)):
    cell = im_gray[label_coords]

    if np.product(cell.shape) < 10:
        print('Dropping label {}. Too small.'.format(label_ind))
        mask = np.where(labels == label_ind + 1, 0, mask)

labels, nlabels = ndimage.label(mask)
print('There are now {} separate components / objects detected.'.format(nlabels))


Dropping label 12. Too small.
Dropping label 13. Too small.
Dropping label 15. Too small.
Dropping label 16. Too small.
There are now 39 separate components / objects detected.


In [9]:
fig, axes = plt.subplots(1, 6, figsize=(10, 6))

for ii, obj_indices in enumerate(ndimage.find_objects(labels)[0:6]):
    cell = im_gray[obj_indices]
    axes[ii].imshow(cell, cmap='gray')
    axes[ii].axis('off')
    axes[ii].set_title('Label #{}\nSize: {}'.format(ii + 1, cell.shape))

plt.tight_layout()
plt.show()

In [10]:
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev + 1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return " ".join([str(i) for i in run_lengths])


print('RLE Encoding for the current mask is: {}'.format(rle_encoding(label_mask)))


RLE Encoding for the current mask is: 30463 2 30717 4 30972 5 31227 6 31482 7 31738 7 31993 8 32249 8 32504 9 32760 9 33016 9 33272 9 33527 10 33783 10 34040 9 34296 9 34552 9 34808 9 35065 8 35321 8 35578 7 35835 6 36092 5 36350 3


In [11]:
import pandas as pd


def analyze_image(im_path):
    im_id = im_path.parts[-3]
    im = imageio.imread(str(im_path))
    im_gray = rgb2gray(im)

    thresh_val = threshold_otsu(im_gray)
    mask = np.where(im_gray > thresh_val, 1, 0)
    if np.sum(mask == 0) < np.sum(mask == 1):
        mask = np.where(mask, 0, 1)
        labels, nlabels = ndimage.label(mask)
    labels, nlabels = ndimage.label(mask)

    im_df = pd.DataFrame()
    for label_num in range(1, nlabels + 1):
        label_mask = np.where(labels == label_num, 1, 0)
        if label_mask.flatten().sum() > 10:
            rle = rle_encoding(label_mask)
            s = pd.Series({'ImageId': im_id, 'EncodedPixels': rle})
            im_df = im_df.append(s, ignore_index=True)

    return im_df


def analyze_list_of_images(im_path_list):
    all_df = pd.DataFrame()
    for im_path in im_path_list:
        im_df = analyze_image(im_path)
        all_df = all_df.append(im_df, ignore_index=True)

    return all_df


In [12]:
testing = pathlib.Path('D:/Projects/starik/stage1_test').glob('*/images/*.png')
df = analyze_list_of_images(list(testing))
df.to_csv('RLE.csv', index=None)

In [13]:
import cv2
import os

In [14]:
TRAIN_PATH = 'D:/Projects/starik/stage1_train'
TEST_PATH = 'D:/Projects/starik/stage1_test'

RANDOM_SEED = 75

CONTOUR_EXTRACT_MODE = cv2.RETR_TREE


In [15]:
train_ids = [x for x in os.listdir(TRAIN_PATH)]
test_ids = [x for x in os.listdir(TEST_PATH)]

In [16]:
df2 = pd.DataFrame({'id': train_ids, 'train_or_test': 'train'})
df2 = df2.append(pd.DataFrame({'id': test_ids, 'train_or_test': 'test'}))

print(df2.groupby(['train_or_test']).count())

               count
train_or_test       
test              65
train            670


In [17]:
df2['path'] = df2.apply(lambda x: 'D:/Projects/starik/stage1_{}/{}/images/{}.png'.format(x[1], x[0], x[0]), axis=1)
print(df2)

                                                count train_or_test  \
0   00071198d059ba7f5914a526d124d28e6d010c92466da2...         train   
1   003cee89357d9fe13516167fd67b609a164651b2193458...         train   
2   00ae65c1c6631ae6f2be1a449902976e6eb8483bf6b074...         train   
3   0121d6759c5adb290c8e828fc882f37dfaf3663ec885c6...         train   
4   01d44a26f6680c42ba94c9bc6339228579a95d0e2695b1...         train   
5   0280fa8f60f6bcae0f97d93c28f60be194f9309ff610dc...         train   
6   0287e7ee5b007c91ae2bd7628d09735e70496bc6127ecb...         train   
7   02903040e19ddf92f452907644ad3822918f54af41dd85...         train   
8   03398329ced0c23b9ac3fac84dd53a87d9ffe4d9d10f1b...         train   
9   03b9306f44e9b8951461623dcbd615550cdcf36ea93b20...         train   
10  03f583ec5018739f4abb9b3b4a580ac43bd933c4337ad8...         train   
11  0402a81e75262469925ea893b6706183832e85324f7b1e...         train   
12  04acab7636c4cf61d288a5962f15fa456b7bde31a021e5...         train   
13  05

In [18]:
from sklearn.cluster import KMeans


def centroid_histogram(clt):
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    hist = hist.astype("float")
    hist /= hist.sum()

    return hist


In [19]:
def get_image_info(path, clusters=2):
    image = cv2.imread(path)
    height, width, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    clt = KMeans(n_clusters=clusters)
    clt.fit(image)
    hist = centroid_histogram(clt)

    bg_idx, fg_idx = 0, clusters - 1
    if hist[bg_idx] < hist[fg_idx]:
        bg_idx, fg_idx = clusters - 1, 0

    bg_red, bg_green, bg_blue = clt.cluster_centers_[bg_idx]
    fg_red, fg_green, fg_blue = clt.cluster_centers_[fg_idx]

    bg_color = sum(clt.cluster_centers_[bg_idx]) / 3
    fg_color = sum(clt.cluster_centers_[fg_idx]) / 3
    max_color_pct = hist[bg_idx]
    min_color_pct = hist[fg_idx]

    return (pd.Series([height, width,
                       bg_red, bg_green, bg_blue, bg_color,
                       fg_red, fg_green, fg_blue, fg_color,
                       hist[bg_idx], hist[fg_idx],
                       fg_color < bg_color]))

In [20]:
names = ['height', 'width',
         'bg_red', 'bg_green', 'bg_blue', 'bg_color',
         'fg_red', 'fg_green', 'fg_blue', 'fg_color',
         'bg_color_pct', 'fg_color_pct', 'invert']

df2[names] = df2['path'].apply(lambda x: get_image_info(x))
df2['shape'] = df2[['height', 'width']].apply(lambda x: '{:04d}x{:04d}'.format(x[0], x[1]), axis=1)


In [21]:
df2.to_csv('conclusion.csv')